In [ ]:
input_test_1 = """###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############"""
output_test_1 = 7036
output_test_1_p2 = 45

input_test_2 = """#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################"""
output_test_2 = 11048
output_test_2_p2 = 64


In [ ]:
import numpy as np
import bisect
from collections import defaultdict

In [ ]:
udlr = {"u" : np.array((-1,0)), "d" : np.array((1,0)), "l" : np.array((0,-1)), "r" : np.array((0,1)) }
rotations = {"u" : "lr", "r" : "ud", "d" : "rl", "l": "du"}

def parse_input(inp):
    g = np.array([[c for c in line] for line in inp.split('\n')])
    p_start = np.argwhere(g == "S")[0]
    p_end = np.argwhere(g == "E")[0]
    g[tuple(p_start)] = "."
    g[tuple(p_end)] = "."
    return g, p_start, p_end

def solve_p1(inp, cost_move = 1, cost_rotate=1000):
    g, p_start, p_end = parse_input(inp)
    #one grid for minimum cost to reach each position for each possible facing
    dict_cost_grids = {c : np.zeros(g.shape) - 1 for c in "udlr"}
    
    #queue contains tuples (cost, position, facing)
    queue = [(0, p_start, "r")]
    
    #min_distance = np.sum(np.abs(p_end - p_start))
    #    if distance < min_distance:
    #        min_distance = distance
    #        print(p)
    
    while queue:
        c,p,f = queue.pop(0)
        distance = np.sum(np.abs(p_end - p))
        if np.all(p == p_end): return c
        dict_cost_grids[f][tuple(p)] = c
        
        #move along facing
        P = p + udlr[f]
        C = c + cost_move
        if g[tuple(P)] == "." and dict_cost_grids[f][tuple(P)] == -1:
            bisect.insort(queue, (c + cost_move, p + udlr[f], f), key=lambda x: x[0]) 
        
        for r in rotations[f]:
            if dict_cost_grids[r][tuple(p)] == -1:
                bisect.insort(queue, (c + cost_rotate, p, r), key=lambda x: x[0])

def key_from_pf(p,f): return (tuple(p), f)                

def solve_p2(inp, cost_move = 1, cost_rotate=1000):
    #similar to solve_p1, except maintain a set of all upstream positions
    g, p_start, p_end = parse_input(inp)
    dict_cost_grids = {c : np.zeros(g.shape) - 1 for c in "udlr"}
    dict_pos_and_f_to_upstream = defaultdict(set)
    
    #queue contains tuples (cost, position, facing), initialized with starting position
    queue = [(0, p_start, "r", None, None)]        
    while queue:
        c,p,f,p_from,f_from = queue.pop(0)
        #if faster path found in meanwhile, skip

        if dict_cost_grids[f][tuple(p)] != -1 and c > dict_cost_grids[f][tuple(p)]: continue
        else:
            dict_cost_grids[f][tuple(p)] = c
            if p_from is not None and f_from is not None:
                dict_pos_and_f_to_upstream[key_from_pf(p,f)].update(dict_pos_and_f_to_upstream[key_from_pf(p_from,f_from)])
                dict_pos_and_f_to_upstream[key_from_pf(p,f)].add(tuple(p_from))

        #Once we're processing costs higher than a path from start to end, we won't discover new shortest paths
        min_cost = max([dict_cost_grids[d][tuple(p_end)] for d in "udlr"])
        if min_cost > -1 and c > min_cost:
            set_all_path_positions = set.union(*[dict_pos_and_f_to_upstream[key_from_pf(p_end, f)] for f in "udlr"])
            set_all_path_positions.add(tuple(p_end))
            return len(set_all_path_positions)
        
        #For forwards or cw/ccw rotations
        p_to_list = [p + udlr[f], p, p]
        f_to_string = f + rotations[f]
        cost_to_list = [c+cost_move, c+cost_rotate, c+cost_rotate]
        
        for p_to, f_to, cost_to in zip(p_to_list, f_to_string, cost_to_list):
            cpf_tuple = (cost_to, p_to, f_to, p, f)
            if g[tuple(p_to)] == "." and dict_cost_grids[f_to][tuple(p_to)] == -1: #if not wall
                bisect.insort(queue, cpf_tuple, key=lambda x: x[0])


In [ ]:
#solvers are slightly slow - inserting elements in sorted queue is expensive
#given input, it seems much faster to treat intersection as nodes instead of advancing tile by tile
for (i,o) in zip([input_test_1, input_test_2], [output_test_1, output_test_2]):
    assert solve_p1(i) == o
solve_p1(input_full)

In [ ]:
for (i,o) in zip([input_test_1, input_test_2], [output_test_1_p2, output_test_2_p2]):
    assert solve_p2(i) == o
solve_p2(input_full)

In [ ]:
input_full = """#############################################################################################################################################
#...........#...........#.....#...........#.........#.....#...#.........#.......#...#.............................#...#....................E#
#.#####.###.#.#.#######.#.#.#.#.#.#######.#.###.#####.###.#.#.#.###.###.###.###.#.#.#.###.#.#.#.#.#####.#########.#.#.###.#.#.###.#########.#
#.#.......#...#.#...#...#.#.#.#.#...#...#.#.#...#.....#.#...#...#...#.........#.#.#...#.#...#.#.#.#.....#.......#...#...#.#.#.#.#.......#...#
#.#.###########.#.#.#.###.#.#.#.###.#.#.#.#.#####.#####.#########.#########.#.#.#.#####.#####.#.#.#######.###.#.#####.#.#.###.#.#######.#.#.#
#.....#...#.....#.#...#...#.#.#.#...#.#...#.....#.#.......#.....#.......#.......#.#.........#...#.............#...#...#.#.#...#.......#...#.#
#.#####.#.#.#####.#########.#.###.###.#########.#.#.#####.#####.#######.#.#.###.#.#######.###############.###.###.#.#.#.#.#.###.#.###.###.#.#
#.#.....#.#.#.....#.........#.....#...#.......#.....#.#...#...#...#...#.#.#...#.#...#...#.....#...#.....#...#.....#...#.#.........#...#...#.#
###.#####.#.#.#####.#.#############.###.#####.###.###.#.###.#.#.#.#.#.#.###.#.#####.#.#.#.#.#.###.#.###.#.#.#.#######.#.#######.#.#####.###.#
#...#.#...#.#.#.....#.#.......#.#...#...#...#.......#...#...#.#.#...#.#.....#.....#.#.#.#.#.......#.#...#.#...#...#...#.......#.#.#...#.#...#
#.###.#.###.#.#######.#.#.###.#.#.###.#####.#######.#.###.###.###.###.###########.#.#.#.#.#.#.#.###.#.###.###.#.#.#.#.#######.#.#.#.#.#.###.#
#...#.......#...#.....#.#...#...#...#.#...#.....#...#.....#.............#.......#.....#.#.#.#.#...........#.....#.#.#.......#...............#
###.#.#####.###.#.#.###.###.#######.#.#.#.###.#.#.#####.###.###.#.#.###.###.###########.###.#.#.#.#####.###.#####.#.#.#######.#.###.#.###.#.#
#...#.#...#.....#.#...#...#.......#...#.#...#.#.#.....#...#.....#.#...#.....#.....#...#.....#.#.#.#...#.#...#...#.#.#.......#.......#.....#.#
#.###.#.#.#.#####.#.#.###.#######.#####.###.#.#######.#.#.#.#.#######.#####.#.###.#.#.#######.#.###.#.###.###.###.#.#.#####.###.###.#######.#
#.#...#.#.#.....#.#.#.....#...............#...#.......#.#...#.......#.#.#...#.#...#.#.........#.....#.........#...#.#.#...#.........#.......#
#.#####.#.#.###.#.#.#######.#.###########.#####.#######.#######.###.#.#.#.###.#.###########.#################.#.###.#.###.#.#####.#.#.#####.#
#.......#...#.#.#.#.......#.#...#.......#.......#.....#...#...#.#.....#...#...#...#.......#.....#.......#...#.#...#.#.....#.#.....#...#...#.#
#.#####.#.#.#.#.#.#.#####.#.###.#.#####.#####.###.#.#####.#.#.#.###.#####.#.#####.#.#####.#####.#.#####.#.#.#####.#.#####.#.#.#####.###.#.###
#...#...#...#...#.#.....#...#...#.#.#...#...#.....#.....#...#.#...#.....#.#...#.#...#...#.#...#.#...#...#.#.......#.#.......#.#.....#...#...#
###.###.#.#.#.#.#.#####.###.#.###.#.#.###.#.###.#####.#######.#.#.#.#.#.###.#.#.#######.#.###.#.###.#.###.#########.#.#####.###.###.#.#####.#
#.#.#...#.#...#.#.....#...#.#...#.#.#.....#.#...#.....#.......#.#.#.#.#...#.#.#.........#...#.#.....#...#.#.....#...#.#...#.....#.....#...#.#
#.#.#.#.#.#.#.#.#####.###.#####.#.#.###.###.#.###.#.###.#########.###.###.###.#.#.###.#####.#.#########.#.#.#.#.#.#.#.#.#########.#####.#.#.#
#.........#.#.........#.#.....#.#.#...#.....#.#...#.#...#.........#...#.......#.#...#.#.....#.........#.#...#.#...#.#...#.........#...#.#.#.#
#.###.#.#.#.###########.#.###.#.#.#.#.#.#####.###.###.###.#.#.#####.#.#########.#.#.###.#####.#####.###.#.###.#######.###.#########.#.###.#.#
#...#.#.#.....#.............#.#.#...#.#.#...#...#.#...#...#.#.#.....#.#...#...#...#.....#.........#...#.#...#.........#.........#...#.#...#.#
###.###.#.#.#.#.#####.#######.#.#####.#.#.#.###.#.#.#######.#.#.#.#####.#.#.###.#####################.#.###.###########.#######.#.#.#.#.###.#
#.......#.....#.....#...#...#.#.....#.#...#.#.#.#.....#.......#.#.#...#.#.#...#.........#.....#...#...#...#.......#.....#...#...#.#.#...#...#
###.###.#.#####.###.#.#.#.#.#.#.#####.#####.#.#.#####.#.#######.###.#.#.#.###.#.#######.#.###.#.#.#.#####.#######.#.#####.#.###.###.#.###.#.#
#.........#.....#...#.#...#.....#.......#...#.#...#.....#.....#...#.#.#.#...#...#...#.....#.#...#.#.............#.......#.#...#.....#.#...#.#
#.#.###.#.#####.#.#.###########.#.#######.#.#.###.#######.###.#.#.#.#.#.###.#####.#.#######.#####.#.###.#######.#.#.#####.###.#######.#.#####
#.........#...#.#.#.....#.....#.#...#.....#.#...#.......#...#.#.....#.....#.......#...#.....#.....#...#...#...#.#...#...#.#...#.......#.....#
###.###.#.#.#.###.#####.#.###.#.###.#.#####.#.#.#######.###.#.#.###########.#########.###.###.#######.#.#.#.###.#####.#.#.#.#######.###.###.#
#.#.....#...#...#.#...#.#...#.......#.#.......#.......#.....#.#.#.........#.#.......#...#.....#.......#.#...#...#...#.#...#...........#...#.#
#.#.###.#.#####.#.#.###.###.#####.###.#.#############.#######.#.#.#####.#.#.#.#.###.###.#.#######.#####.###.#.###.#.#.###############.#.###.#
#.#.#...#.....#.#.#...#...#.#...#...#.#.#...#.......#.....#...#.#.#.......#.#.#.#...#...#.......#.....#.....#.#...#.#...#.#.....#.....#.#...#
#.#.#.###.###.#.#.###.#.###.#.#.#####.#.#.#.#####.#.#####.#.#####.#.#######.#.#.###.#.#########.#########.#.#.#.###.###.#.#.#.###.#######.#.#
#.#.#...#.#.....#.....#.....#.#.......#...#.......#...#...#.#.....#.#.....#.#.#...#.#...#.......#.......#.#.#.#.#.#.#.#.#...#.............#.#
#.#.###.#.#####.#####.#######.#######.###############.#.###.#.#####.#.#.#.#.#####.#.###.#.###.###.#####.#.#.#.#.#.#.#.#.#.#################.#
#.....#.#.....#.....#...#...#.......#.#.........#...#.#...#.#...#.#.#.#.#.#.#.....#.....#...#.....#.....#...#...#.#...#.#.#...........#...#.#
#.#.###.#####.#########.###.#######.###.#######.###.#.###.#.###.#.#.###.###.#.###.#.#######.#.#####.#####.#.#####.#.#.#.###.#########.###.#.#
#.#.........#...#.....#.#.........#...#.....#...#...#.#.#.#.....#.#...#...........#.#.....#...#...#.#...#.#...#.....#...#...#.......#...#...#
#.#.#.#####.###.#.#.###.#.#####.#.###.#####.#.###.#.#.#.#.#######.###.#######.#####.#.###.#######.#.#.#.###.###.#####.#.#.#########.###.#####
#.#.....#.#...#...#...#.#.#...#.#...#.....#.#...#.#.#.#.#...........#.#.....#.#...#.#.#...........#.#.#...#.........#.......................#
#.###.#.#.###.#######.#.#.#.#.#.#.#.#####.#.###.#.###.#.#########.#.#.#.###.###.#.#.#.#.###.#.#####.#.###.###.###############.#.#####.#####.#
#...#...#...#...#...#.#...#.#.#...#.#...#.....#.#.....#...#.....#.#...#.#.#.....#...#.#...#.#.......#...#...#...#...#.......#.#.#...#.#...#.#
#.#.#.###.#####.###.#.#####.#.#####.###.#######.#.#####.###.###.#.###.#.#.###########.###.###.#.#####.#.###.#####.#.#.#####.#.#.#.#.#.#.###.#
#.#...#...#...#...#.#...#...#.#...#.#...#.......#.......#...#...#...#...#...#...#.#...#.....#.#.#.....#.#.....#...#...#.#...#...#.#.....#...#
#.###.#.###.#.###.#.###.#.###.#.#.#.#.#.#.###.#####.#####.###.#####.#.###.#.#.#.#.#.###.###.#.#.#.#####.#####.#.#######.#.###########.###.#.#
#.............#...#.....#...#...#.#...#.....#.......#.....#.#.....#...#...#...#...#.#.....#.#.#.#.#...#.#...#...#.......#.#...#...#.....#...#
#.#.#.#.#####.#.###.#.#.###.#####.#####.###.###.#.###.#.###.###.#.#####.#########.#.#.#.#.#.#.#.#.###.#.#.#.###.#######.#.#.#.#.#.#.#.#.#.#.#
#.#.#.....#...#.#...#.#...#.....#...#.....#.#...#...#.#.......#.#...............#.#...#.#.#.#.#.......#...#...#.#.....#.....#.#.#...#...#.#.#
#.#.#.#####.#.#.#.#.#.###.###.###.#.#.#####.#.#####.#.#.###.###.#.#############.#######.#.#.#.#####.#########.###.###.#######.#.#####.#####.#
#.#.#.#.....#.#.#.....#...#...#...#.#.......#.#...#...#...#.#...#.#...#.....#.#.........#.#.#.....#.#.......#.....#.#...#...#...#...#.#.....#
#.#.#.#.###.###.###.#.#.###.###.#####.#.#####.#.#.#######.#.#.###.#.#.#.#.#.#.#.#.#######.#.#####.#.#.#####.#.#####.###.#.###.###.#.#.#.###.#
#.#...#.........#...#.#.#...#.#.........#...#.#.#.#.......#.#.#.....#.#.#.#...#...#.....#.#.#.....#.#.#...#.#.......#...#.....#...#.#.#.#...#
#.#.#.#####.#####.###.#.#.###.#########.#.#.#.#.#.#.#.#####.#.#######.#.#.###.#.###.###.#.#.#.#######.###.#.#.#######.###.###.#.#.###.#.#.#.#
#.#.#...#.........#...#...#...........#.#.#.#.#.#.#...#...#.#...#...#.#.#.#.#.#.........#.#...#.......#...#.#.#.....#.#...#...#.#.....#...#.#
#.#.#.#.#.#.#####.#.#########.###.###.#.#.#.#.#.#.#.###.#.#.###.#.#.#.###.#.#.#########.#.#####.###.###.#.#.#.#.###.#.###.#.#.#.#####.#####.#
#.#.#.#...#.....#.#...#.....#...#...#.#.#.#...#.#...#...#.....#.#.#.#.....#.#.....#.....#.#.#...#.#.#...#...#.#...#.#...#.#.#...#.........#.#
#.#.#.#.###.#.#.#.###.#.###.#######.#.#.#.###########.#.#######.#.#.#######.#####.#.###.#.#.#.###.#.#.#####.#.###.#.###.#.#.###.###.#.###.#.#
#.....#.....#.#.#.#...#...#.....#...#.....#...........#.........#.#.....#...#.....#.#...#.#...#...#.#.#.....#...#.#.....#.#...#...#.#.....#.#
#.###.#####.###.#.###.###.#.###.#.#########.###########.#.#.###########.#.#.#.#####.#.###.#.###.###.#.#.###.###.#.#######.###.###.#.#.#.###.#
#...#.#...#.....#...#...#...#...#.#.........#...#.....#.#...#.#.........#.#.#.#.....#...#.#.....#...#.#.#...#...#...#.....#.#.....#...#.....#
#.#.#.#.#.#.###.###.###.###.#.###.#.#.#######.#.#####.#.###.#.#.#.###.#.#.#.#.#.#########.#######.###.#.#.#.#######.###.#.#.#########.#####.#
#.......#...#...#.#.#.#.....#.#...#.#.....#...#.#.....#...#.#...#.#...#...#...#...........#.......#...#...#.......#...#.#...........#.......#
###.#######.###.#.#.#.#######.#.###.#####.#.###.#.#######.#.#####.#####.#####.#########.###.#######.#.###########.###.#.#.#.#######.#.#.###.#
#...#.....#...#...#.#.....#...#...#.#...#...#.#.#.........#.....#...#...#.............#.....#.......#.........#.#.#...#.#.#.#.....#...#...#.#
#.#.#.#####.#.#####.#.#.###.###.#.#.###.#####.#.#.#####.#######.#.#.#.#.#.###########.#.#####.###.###.###.###.#.#.#.###.###.#.#.#####.#.###.#
#.#.................#...#...#...#...........#.#.......#.#...#.....#.......#.......#.#...#.....#.#...#.#...#...#.#...#...#...#.........#.#...#
#.#.#.#####.###.#######.#.#.#.#######.#####.#.#######.#.#.#.#.#####.#########.###.#.#####.#####.###.###.###.###.#####.###.#######.#.#.###.###
#...#.#.......#.#.....#.#...#.#...#.......#.....#.....#...#.#.#...#...#.#.....#...#.......#.......#.....#.#.........#.....#.....#.#.#...#...#
###.#.#.#####.#.#.###.#.###.###.#.#####.#.#.#####.#######.###.#.#.#####.#.#####.###.###.###.#####.#######.###.###.#.#######.###.#.###.#.#.#.#
#...#.#.....#.....#...#.....#...#.......#.#.#...#.#...#.#.#...#.#.....#.#...#.....#.#.......#...#.................#.....#...#.#.#.#...#.#.#.#
#.###.#####.#######.###.#.###.###########.#.#.#.#.#.#.#.#.#.###.#####.#.#.#.#####.#.#.#######.#.###.#######.###########.###.#.#.#.#.#.#.###.#
#.#...#.......#.....#...#.#.......#.......#.#.#.#.#.#...#.#.....#...#...#.#...#.....#.........#...#.........#.......#.#.#...#.#...#.#.#.#...#
#.#####.#.#####.#####.###.#.#####.#####.###.#.#.#.#.#####.#########.#####.###.#####.#.#####.#.###.#######.###.#####.#.#.#.###.###.#.#.#.#.#.#
#...........#...#.#...#.........#.....#.#.#.#.....#.#...............#...#...#.......#.#...#.#.#...#.......#...#.....#.#...#.......#.#...#.#.#
#############.###.#.###.#############.#.#.#.#.#####.#.#############.#.#.###.#######.###.#.###.#.#######.###.###.#####.#####.###.###.#.###.#.#
#.....#.......#...#...#.#.....#.......#.#.#.#.#.......#...#.....#.#...#...#...#...#.....#.#...#.......#.#...#...#.......#...#.....#.#...#.#.#
#.###.#.#######.#####.###.###.#.#######.#.#.#.###.#####.###.###.#.###.###.###.#.#########.#.#########.#.#.###.###.#.#.#.#.###.#.#.#.#.#.#.#.#
#...#.#.#.....#.......#...#...#.#.....#.#...#...#.#.....#...#...#.......#.#...#.....#...#...#.........#.#.#.#.....#.#.#.#.#.#.#.#.#.#.#...#.#
#.#.#.#.#.#.#.#.#######.###.###.#.###.#.#.#####.#.###.###.###.###.#####.#.#.#####.#.###.#####.#.#########.#.#######.#.###.#.#.#.#.#.#.#####.#
#.#.#...#.#...#.........#...#...#...#...#.....#.#.#...#...#.#...#.....#...#.......#.........#.#.#.........#.#.......#.....#.....#.#...#...#.#
#.#.#####.###.#####.#.###.###.#####.###########.#.#.#.#.###.###.#.#.#.#######.#############.#.#.#.#########.#.#######.#####.###.#.###.###.#.#
#.#...#.#.#.#...#...#.#.#.#...#.....#...........#...#.#...#.#...#.#.#.........#.....#.....#.#.#.#.#.........#.#.......#.....#...#.......#.#.#
#.###.#.#.#.#.#.#.###.#.#.###.#.#.#.#.###############.#.#.#.#.###.#.###########.###.#.###.#.#.###.#####.#.###.#.#############.#.#####.#.#.#.#
#.#.#.........#.#.#...#.#.....#.....#.........#.....#.#.#...#...#.#.#...#.......#...#...#...#.#.......#.#.#...#.....#.......#.#.....#.#...#.#
#.#.#.#####.#.#.#.###.#.#######.#####.#######.###.#.#.#.###.###.###.#.#.#.#######.#.#.#.#####.#.###.#.#.###.#######.#.#####.#.#####.#.###.#.#
#...#.....#...#...#...#.......#.....#.......#...#.#.#...#...#.#.......#.#.......#...#.#.....#...#...#.#...#...#.....#...#...#.#...#...#...#.#
#.#.#.###.###.###.#.#####.###.###.#.#.###.#####.#.#.###.#.###.###########.#####.###.###.###.#.###.###.###.###.###.#####.#.###.#.###.#######.#
#...#...#.......#...#.....#...#.....#...#.#...#.#.......#.#...#.....#...#.#.....#...#...#.#.#.#...#.........#...#.......#.....#.#...#.....#.#
#.###.#.#.###.#.#######.###.###.#####.#.###.#.#.#.#######.#.###.#.#.#.#.###.#.###.###.###.#.#.###.#.#.#######.#.###############.#.###.###.#.#
#.....#.#.....#...#...#...#.#...#.....#.....#...#.....#...#.....#.#.#.#.....#.......#.#...#.#.....#.#.......#.#...#...#.........#...#.#.#.#.#
#.###.#.#.#.#.###.#.#.###.#.#.###.###.#############.#.#.###.#####.#.#.#############.#.###.#.#########.#####.###.#.###.#.###.#######.#.#.#.#.#
#.....#.#...#.....#.#.........#.#...#.#...#.........#.#.#.....#...#.#.#.........#...#...#.#...........#...#...#.#...#.#...#...#.....#.#.#...#
#.#.#.#.#.#.#.#####.#######.#.#.#.#.#.#.#.#.#########.#.#.#.###.###.#.#.#######.###.###.#.###########.#.#.###.#.###.#.###.#.#.#.#####.#.#####
#.#.#...#.#.#.......#.....#.#.#...#.#.#.#...#...#.#...#.#.#.....#.#.#...#.....#...#.....#...#.....#.....#...#.#.#...#.#.....#.....#...#...#.#
#.#.#.###.#.#.###.###.###.#.#.#.###.###.#####.#.#.#.###.###.#####.#.#########.###.#######.#.###.#.#.#.#######.###.###.#.###########.#.#.#.#.#
#...#.....#.....#.....#.#.#.#.#...#.#...#...#.#...#.....#...#.#...#.#...........#.#...#...#.....#.#.#...#...#.#...#.........#.......#...#...#
#.#.#.###.#.###.#.#####.#.###.###.#.#.###.#.#.###.###.#.#.###.#.#.#.#.#######.#.#.#.###.#.#####.###.###.#.#.#.#.#.#######.#.#.#######.#.###.#
#.#.....#...#.#.#.......#...#...#.#.#.#...#...#.....#.#...#...#.#...#.......#.#.#.#.#...#...#...#...#.#...#.#.#.#.#.......#...#.#.....#...#.#
#.###.###.#.#.#.#########.#.###.###.#.#.#.#.#######.#.#####.#.#.#####.#.#.#.#.#.#.#.#.#####.#.###.###.###.#.#.#.###.###########.#.#######.#.#
#.......#.#...#...#.......#...#.....#.....#.#.....#.#.....#.#...#.....#.#.#.#...#.#.#...#...#.#...#.......#...#...#.#.......#...#.#.....#.#.#
#####.#.#.#.#####.#.#####.#.#.#.#########.###.###.#######.#.#####.###.#.#.###.#.#.#.#.###.#.###.###.#.###.#####.#.#.#.###.#.#.#.#.###.#.#.#.#
#.......#.#.....#.#.....#.#.#...#.......#.#...#.#.#...#...#.#...#...#.#.#.......#.#...#...#.#...#...#...#.....#...#.#.#...#.#.#.#.#...#.#.#.#
#.###.###.#.#.###.#####.#.#######.#.#####.#.###.#.#.#.#.#.#.#.#.#####.#.###.#.###.#.###.###.#.#######.#.#####.#.###.###.###.#.#.#.#.###.#.###
#.#.....#.#.#...#...#...#...#.....#.......#.#...#...#...#.#.#.#...#...#.#.#.#.#...#.#...#...#.......#.#.....#.#...#...#...#.#.#.#...#...#...#
#.#.#.#.#.#.###.#.#.#.#####.#.#############.#.#.#########.#.#.###.#.###.#.#.#.#.###.#.#############.#####.#.#.###.###.###.#.#.#.#####.#####.#
#.#...#.#.#...#.....#.#...#.#...#.........#.#.#...#.......#.....#...#.#...#.#.#.#...#.#.....#.....#.....#.#.#.......#.#...#...#.........#...#
#.#.#.#.#.#.###.###.#.###.#.###.#.#########.###.###.#######.#.#.#####.###.#.#.#.#.###.#.###.#.#.#.#.###.###.#####.###.#.#####.###########.###
#...#.#.#.........#...#...#.#...#...#.....#...#.....#...#.#.#.#.....#.....#.#.#.#...#...#.#...#.#.....#.....#...#.#...#.........#...#...#...#
#.#.#.#.###.#.###.#####.###.#.#####.#.###.###.#.#####.#.#.#.#######.#.#####.#.#.###.#####.#####.###.#########.#.#.#.###.#########.#.#.#.#.#.#
#.#.#.#...#...#...#.......#...#...#.#.#...#...#.#.....#.........#...#.....#...#.#.........#...#.#...#.......#.#...#.#.....#...#...#.........#
#.#.#.#.###.###.#.#######.###.#.#.#.#.#.###.###.#.#.###########.#.#######.#####.#######.#.#.#.#.#####.#####.#.#####.#.#####.#.#.#########.#.#
#.#.#.#...#...#.#.......#.......#.#...#...#.#...#.#.#.....#.....#.#.....#.....#...#.......#.#.#.......#.....#.#.....#.#.....#...#.........#.#
#.#.#.###.###.#.#######.#######.#.#######.#.###.#.###.###.#.#####.#.###.#########.###.#####.#.#####.###.#####.#.#######.#########.###.###.#.#
#.#.#.#.#.....#.#.#...#...#.....#.........#...#.#...#.#.#...#...#...#...........#...#.#.....#.....#...#.......#.........#.........#...#.....#
#.#.#.#.#######.#.#.#.###.#.#############.###.#.#.#.#.#.#####.#.#.###.#########.###.###.#######.#.#.#####################.#######.#.#.#.#.###
#.#...#...#.....#...#...#.#.#...............#.#.#.#.#.......#.#.#...#.......#.....#.....#.....#.#.#.#...#...............#.#.........#.#.#...#
#.#.###.#.###.#.#####.###.#.#.###.#.#######.#.#.###.#######.#.#.###.#######.#.###########.#.#.#.#.###.#.#.###.#########.#.#.#.#####.###.#.###
#.#.....#...#.#.....#...#...#.#...#.#.......#.#...#.#.....#...#...#...................#.#.#...#.#.....#...#...#...#...#.#.........#.#...#...#
#.###.#####.#.#####.###.#######.###.#.#######.###.#.#.#.#########.#######.#########.#.#.#.#.###.###########.###.#.#.#.#.###########.#.#####.#
#.....#...#.#.....#...#.#.......#...#.#.#.....#...#.#.#.....#.....#.....#.#.......#.#.#.#.#...#.....#...#...#...#...#.#...#.....#...#.#.....#
#.###.#.#.#.#####.###.#.#.#.#########.#.#.###.#.###.#.###.###.#####.#.#.#.#.#####.#.#.#.#.###.#.###.#.#.#.###.#######.#.#.#.###.#.###.###.#.#
#.......#.#.........#.#.#.#.....#.............#.....#.#.#.#...#.......#.#.#.#.......#...#.#.....#.....#.#...#.......#.#.#.#...#...#...#...#.#
#.###.###.#.#.#######.#.#.#####.#.###########.#######.#.#.#.#######.#.#.#.#.#######.#####.#####.#####.#####.###.#####.#.#.#.#.#.###.###.###.#
#.#.......#.#.#...#...#...#...#...#.#.........#.........#.#...#...#.#.#.#.#.........#...#...#.................#.#.....#.......#.#...#...#.#.#
#.#.#.#####.#.#.#.#.###.###.#.#.###.#.#.#####.#.#########.###.#.#.###.#.###.#######.#.#.###.###.#####.#######.#.#.#####.#######.#.###.###.#.#
#.#.#...#...#.#.#...#.#.....#...#...#.#.......#.#...#.....#.#...#.....#...#.#.....#...#...#.....#.......#.....#.#.#.......#.................#
#.#.#.#.#.###.#.#####.#######.#.###.#.#.#######.###.#.###.#.#.###########.#.###.###.###.#.#####.#.###.#.#.#####.#.#.###.#.#.#####.#.#.#######
#.#...#.#...#.#.#.....#.........................#...#.#.....#...#.......#.........................#...#.#.#.....#.#.#.#...#.................#
#.###.#.###.###.###.#.#.#.###.###.#.###.#########.#.#.#####.#.#.#.#####.#.###.#.#.###.###.###.#.###.#.#.#.#.#####.#.#.#.#####.#.#.#####.###.#
#.....#...#.#...#...#.#...#.....#.#.............#.#.#.....#...#.#...#...#...#...#...#...#.#...#.....#.#...#.#...#...#.#.#...#...#.#...#.#...#
#.###.###.#.#.###.###.#.#####.#.#.###.#.#####.#.#.#.#####.#.#.#.###.#.#####.#######.###.#.#.###.#####.###.#.#.#.###.#.#.#.#####.#.#.#.###.#.#
#.#...#...#.#.#...#.....#.....#.#...#...#.....#...#...#...#.....#...#.....#...#.....#.#...#.#...#...#...#.#...#.......#.#.......#.#.#.....#.#
#.#.#.#.#.#.#.#.#########.###.#####.#.#.#.#########.###.###.#.###.#####.#####.#.#####.#.###.#.#.#.#.#.#.#.#########.#.#.#########.#.#######.#
#.#.....#.#...#...........................#.......#.#...#.........#...#.......#.#.....#.#...#.....#...#.#.#...........#...#...#...#.......#.#
#.###.#########.###.#.#.#.#.#.#.#######.###.#.#####.#.###.#.#######.#.#########.#.###.#.#.#####.#####.#.###.#######.###.#.#.#.#.#####.###.#.#
#S..........................#...........#...........#...............#...........#.......#.............#.................#...#.............#.#
#############################################################################################################################################"""